<a href="https://colab.research.google.com/github/goyoh/arena/blob/master/%E3%82%AD%E3%83%BC%E3%83%AF%E3%83%BC%E3%83%89%E3%81%B8%E3%81%AE%E3%82%AB%E3%83%86%E3%82%B4%E3%83%AA%E8%87%AA%E5%8B%95%E5%89%B2%E3%82%8A%E5%BD%93%E3%81%A6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# -*- coding: utf-8 -*-
# Google Colabで実行することを想定

# 1. 必要なライブラリをインストール
# gspread: Google Sheets APIを操作
# pandas: データ操作 (ここでは主にデータフレームでの整理に利用可能だが、必須ではない)
# sentence-transformers: テキストの埋め込みベクトル生成と比較
# Google認証関連ライブラリ
!pip install --upgrade google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client gspread pandas sentence-transformers -q

print("ライブラリのインストールが完了しました。")

# 2. Google Driveのマウントと認証
# ColabからGoogle DriveやGoogle Sheetsにアクセスするために必要
from google.colab import auth
try:
    auth.authenticate_user()
    print("Googleアカウント認証が成功しました。")
except Exception as e:
    print(f"Googleアカウント認証に失敗しました: {e}")
    print("ノートブックの設定でサードパーティのCookieが許可されているか確認してください。")
    raise # エラーで停止

import gspread
from google.auth import default
import re
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch
import numpy as np

# Google APIクライアントの認証情報を取得
try:
    creds, _ = default()
    gc = gspread.authorize(creds)
    print("Google Sheets APIへのアクセス準備ができました。")
except Exception as e:
    print(f"Google Sheets APIの認証に失敗しました: {e}")
    raise # エラーで停止

# 3. スプレッドシート情報とカテゴリリスト、モデルの設定
# --------------------------------------------------
# ▼▼▼ ユーザー設定箇所 ▼▼▼

# 対象のスプレッドシートURL (必ず編集してください)
# 例: 'https://docs.google.com/spreadsheets/d/XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX/edit#gid=0'
SPREADSHEET_URL = 'https://docs.google.com/spreadsheets/d/1bFlIif-Uu7Df2jdSFqY8uqLbLDE36zvz1jFOvmD-6QI/edit?gid=0#gid=0'

# 対象のシート名 (シート名が異なる場合は編集してください)
SHEET_NAME = 'シート1'

# 関連性を判定したいカテゴリのリスト (自由に編集・追加してください)
# 例: 100種類程度まで
categories = [
    'AI', 'BEAM Plus Certification', 'BEMS (Building Energy Management System)', 'BREEAM Certification', 'EDGE Green Building Certification', 'ESG Management Support', 'LEED Certification', 'WELL Certification', 'Accessibility', 'Facilities for active commuting', 'Initiative participation', 'Event', 'Eco-friendly temperature control equipment', 'Eco-friendly lighting equipment', 'Eco-friendly transportation options', 'Eco-friendly air conditioning equipment', 'Energy consumption', 'Engagement Program', 'All electric', 'Guideline', 'Green lease clause in rental contracts', 'Community-oriented services', 'Concierge Service', 'Stress-relief spaces', 'Obtaining technology-related certifications', 'Digital Connectivity', 'Heat Island Reduction', 'Business Facility', 'Building Management System', 'Fitness', 'Universal design', 'Recycle facility', 'Religious Diversity', 'Individual temperature control', 'Provision of healthy menu options', 'Tenant-controlled lighting', 'Public Transportation', 'Renewable energy', 'Health Centres', 'Smoking restrictions', 'Soil contamination asbestos PCB etc.', 'Promotion of community engagement', 'Opening spaces to local residents', 'Vertical Transportation', 'Green Facade', 'Building envelope performance and solar heat shielding', 'Solar Panel', 'Donation', 'Rooftop terrace', 'Rooftop greening', 'Indoor greening', 'store', 'Waste management', 'Building safety', 'Environmental considerations during construction', 'Construction Material', 'Daylight harvesting', 'Ventilation and air cleanliness', 'Human Development', 'Construction work for extending building lifespan', 'Water usage', 'Stormwater attenuation', 'Thermal comfort', 'Humidity control', 'Environmental certification acquisition', 'Consideration for ecosystem preservation', 'Smoke-free environment', 'Air Quality Visualization', 'Air Quality Improvement', 'Water-saving facilities', 'Management system', 'Greening', 'Access to natural environments', 'Hygienic Facility', 'Design', 'Procurement', 'Spaces for physical activity', 'Promotion of physical activity', 'Heritage Preservation and Retrofit', 'Urban farm', 'Security Facility', 'Sound absorption performance', 'Rainwater Reuse', 'Emergency elevator', 'Emergency supplies', 'Emergency stockpile', 'Emergency Power Supply', 'Non-potable water reuse', 'Acoustic equipment', 'Food loss reduction', 'Onsite Food Production', 'Car Parking', 'Bike Parking',
    # 必要に応じてさらにカテゴリを追加
]

# 関連性の閾値 (0.0から1.0の間で設定)
# この値以上の類似度スコアを持つカテゴリのみが割り当てられます。
# 低い値 (例: 0.3) にすると、関連性が低くても何かしらのカテゴリが割り当てられやすくなります。
# 高い値 (例: 0.6) にすると、関連性が明確な場合にのみカテゴリが割り当てられます。
SIMILARITY_THRESHOLD = 0.4

# 使用するSentence Transformerモデル名
# 日本語を含む多言語対応モデル or 日本語特化モデルを選択
# 例:
# 'paraphrase-multilingual-mpnet-base-v2' (多言語、汎用的)
# 'cl-tohoku/bert-base-japanese-whole-word-masking' (日本語特化、BERTベース)
# 'sonoisa/sentence-luke-japanese-base-lite' (日本語特化、LUKEベース、比較的軽量)
# 'intfloat/multilingual-e5-large' (多言語、高性能だがリソース要求大)
MODEL_NAME = 'paraphrase-multilingual-mpnet-base-v2'

# ▲▲▲ ユーザー設定箇所 ▲▲▲
# --------------------------------------------------

# --- 処理開始 ---

# スプレッドシートIDをURLから抽出
spreadsheet_id = None
match = re.search(r'/d/([a-zA-Z0-9-_]+)', SPREADSHEET_URL)
if match:
    spreadsheet_id = match.group(1)
    print(f"スプレッドシートID: {spreadsheet_id} を抽出しました。")
else:
    print(f"エラー: 有効なGoogle スプレッドシートのURL形式ではありません。URLを確認してください: {SPREADSHEET_URL}")
    raise ValueError("無効なスプレッドシートURLです。")

# 4. Sentence Transformerモデルの読み込み
try:
    print(f"Sentence Transformerモデル '{MODEL_NAME}' を読み込んでいます... (初回は時間がかかることがあります)")
    # GPUが利用可能ならGPUを使用
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = SentenceTransformer(MODEL_NAME, device=device)
    print(f"モデル '{MODEL_NAME}' の読み込みが完了しました。(使用デバイス: {device})")
except Exception as e:
    print(f"エラー: モデル '{MODEL_NAME}' の読み込みに失敗しました: {e}")
    print("モデル名が正しいか、Colabのランタイムタイプ（GPUが推奨）を確認してください。")
    raise # エラーで停止

# 5. カテゴリリストのベクトル化 (処理効率化のため事前に計算)
try:
    print("カテゴリリストをベクトル化しています...")
    with torch.no_grad(): # 計算グラフを作成しないモードでメモリ効率化
        category_embeddings = model.encode(categories, convert_to_tensor=True, show_progress_bar=True, device=device)
    print(f"カテゴリリスト ({len(categories)}件) のベクトル化が完了しました。")
except Exception as e:
    print(f"エラー: カテゴリのベクトル化中にエラーが発生しました: {e}")
    raise # エラーで停止

# 6. スプレッドシートを開き、データを読み込む
try:
    print(f"スプレッドシート '{spreadsheet_id}' を開いています...")
    spreadsheet = gc.open_by_key(spreadsheet_id)
    worksheet = spreadsheet.worksheet(SHEET_NAME)
    print(f"シート '{SHEET_NAME}' へのアクセスに成功しました。")

    # A列のデータを取得 (ヘッダー行を除く場合や、特定の範囲を指定する場合は調整)
    # worksheet.col_values(1) はA列全体を取得
    # 空のセルを除外してリスト化
    keywords_list = [cell for cell in worksheet.col_values(1) if cell and cell.strip()] # A列の値を取得し、空白セルやスペースのみのセルを除外

    if not keywords_list:
        print("A列に有効なキーワードが見つかりませんでした。処理を終了します。")
    else:
        print(f"{len(keywords_list)} 件のキーワードをA列から読み込みました。")

        # 7. 各キーワードに対して関連カテゴリを判定
        results_list = []
        print("各キーワードとカテゴリの関連性計算を開始します...")

        # キーワードをバッチ処理でベクトル化 (大量データの場合に効率的)
        print("キーワードをベクトル化しています...")
        try:
            with torch.no_grad():
                 keyword_embeddings = model.encode(keywords_list, convert_to_tensor=True, show_progress_bar=True, device=device)
            print("キーワードのベクトル化が完了しました。")
        except Exception as e:
             print(f"エラー: キーワードのベクトル化中にエラーが発生しました: {e}")
             raise

        print("類似度を計算し、最適なカテゴリを選択しています...")
        for i, keyword_emb in enumerate(keyword_embeddings):
            # キーワードベクトルと全カテゴリベクトルのコサイン類似度を計算
            # cos_simは [-1, 1] の範囲のテンソルを返す。形状は [1, num_categories]
            cos_scores = util.cos_sim(keyword_emb.unsqueeze(0), category_embeddings)[0] # keyword_embを2Dテンソルにする

            # 最も類似度が高いカテゴリのインデックスとスコアを取得
            best_match_idx = torch.argmax(cos_scores).item()
            best_match_score = cos_scores[best_match_idx].item()

            # 閾値と比較して割り当てるカテゴリを決定
            assigned_category = '' # デフォルトは空白
            if best_match_score >= SIMILARITY_THRESHOLD:
                assigned_category = categories[best_match_idx]
                print(f"  - キーワード: '{keywords_list[i]}' -> カテゴリ: '{assigned_category}' (スコア: {best_match_score:.4f})")
            else:
                # 閾値未満の場合は割り当てない
                print(f"  - キーワード: '{keywords_list[i]}' -> カテゴリ: なし (最高スコア: {best_match_score:.4f} < 閾値: {SIMILARITY_THRESHOLD})")

            results_list.append([assigned_category]) # gspreadのupdate用にリストのリスト形式にする

        # 8. 結果をスプレッドシートのB列に書き込む
        if results_list:
            # 書き込み開始セルを指定 (例: 'B1'から)
            # ヘッダー行がある場合は 'B2' からにするなど調整してください
            # 例: start_cell = 'B2'
            start_cell = 'B1'
            end_row = len(results_list) # ヘッダーなしの場合
            # end_row = len(results_list) + 1 # ヘッダーありの場合

            range_to_update = f'{start_cell}:B{end_row}'

            print(f"\n計算結果をスプレッドシートの範囲 '{range_to_update}' に書き込みます...")
            try:
                worksheet.update(range_to_update, results_list, value_input_option='USER_ENTERED')
                print("スプレッドシートへの書き込みが完了しました。")
            except gspread.exceptions.APIError as e:
                 print(f"エラー: スプレッドシートへの書き込み中にAPIエラーが発生しました: {e}")
                 print("書き込み権限があるか、APIの割り当て制限に達していないか確認してください。")
                 raise
            except Exception as e:
                 print(f"エラー: スプレッドシートへの書き込み中に予期せぬエラーが発生しました: {e}")
                 raise
        else:
            print("書き込む結果がありませんでした。")

except gspread.exceptions.SpreadsheetNotFound:
    print(f"エラー: スプレッドシートが見つかりません。ID '{spreadsheet_id}' が正しいか、アクセス権があるか確認してください。")
except gspread.exceptions.WorksheetNotFound:
    print(f"エラー: シート '{SHEET_NAME}' が見つかりません。スプレッドシート内にその名前のシートが存在するか確認してください。")
except Exception as e:
    print(f"予期せぬエラーが発生しました: {e}")
    import traceback
    traceback.print_exc() # 詳細なエラー情報を表示

print("\n--- 全ての処理が終了しました ---")

ライブラリのインストールが完了しました。
Googleアカウント認証が成功しました。
Google Sheets APIへのアクセス準備ができました。
スプレッドシートID: 1bFlIif-Uu7Df2jdSFqY8uqLbLDE36zvz1jFOvmD-6QI を抽出しました。
Sentence Transformerモデル 'paraphrase-multilingual-mpnet-base-v2' を読み込んでいます... (初回は時間がかかることがあります)
モデル 'paraphrase-multilingual-mpnet-base-v2' の読み込みが完了しました。(使用デバイス: cpu)
カテゴリリストをベクトル化しています...


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

カテゴリリスト (93件) のベクトル化が完了しました。
スプレッドシート '1bFlIif-Uu7Df2jdSFqY8uqLbLDE36zvz1jFOvmD-6QI' を開いています...
シート 'シート1' へのアクセスに成功しました。
6 件のキーワードをA列から読み込みました。
各キーワードとカテゴリの関連性計算を開始します...
キーワードをベクトル化しています...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

キーワードのベクトル化が完了しました。
類似度を計算し、最適なカテゴリを選択しています...
  - キーワード: 'High-Speed Internet' -> カテゴリ: 'Digital Connectivity' (スコア: 0.6876)
  - キーワード: 'Advanced Toilets and Restrooms' -> カテゴリ: 'Hygienic Facility' (スコア: 0.6427)
  - キーワード: 'Dining Area' -> カテゴリ: 'Indoor greening' (スコア: 0.4812)
  - キーワード: 'Soundproof Phone Booths' -> カテゴリ: 'Sound absorption performance' (スコア: 0.6075)
  - キーワード: 'Seismic Design' -> カテゴリ: 'Construction Material' (スコア: 0.4992)
  - キーワード: 'Business Continuity Plan (BCP)' -> カテゴリ: 'Business Facility' (スコア: 0.6032)

計算結果をスプレッドシートの範囲 'B1:B6' に書き込みます...
スプレッドシートへの書き込みが完了しました。

--- 全ての処理が終了しました ---


<ipython-input-3-19d15bc1a230>:176: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(range_to_update, results_list, value_input_option='USER_ENTERED')
